In [1]:
import slack
import openai
import os
from pathlib import Path
from dotenv import load_dotenv
from flask import Flask
from slackeventsapi import SlackEventAdapter
import pandas as pd
df = pd.read_csv('QandA_list.csv', encoding='utf-8')
answers = df["Question answers"].tolist()
env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)

openai.api_key = token=os.environ['CHAT_TOKEN']

In [7]:
def gpt_test(prompt):
	response = openai.ChatCompletion.create(
		#Choose model
		model = "gpt-3.5-turbo-0613",
		#Define 
		messages = [
			{"role": "system", "content" : "The user wants to learn math, keep them on the topic of math whenever possible and nudge them in the direction of math whenever they veer off course"},
			{"role": "user", "content": prompt},
		],
		max_tokens = 500,
        temperature = 0.8,
	)
	return response

[<OpenAIObject at 0x7f7007fc78d0> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "Of course! I'd be happy to help you learn linear algebra. Linear algebra is a branch of mathematics that deals with vector spaces and linear transformations. It involves the study of systems of linear equations, matrices, vectors, and their properties.\n\nTo start, let's talk about vectors. A vector is a quantity that has both magnitude and direction. It can be represented geometrically as an arrow in space. In linear algebra, vectors are typically represented as column matrices. For example, the vector [1, 2, 3] can be represented as:\n\n| 1 |\n| 2 |\n| 3 |\n\nVectors can be added together, and scalar multiplication can be applied to them. Addition of vectors is done by adding corresponding entries. Scalar multiplication is multiplying each entry of the vector by a scalar value.\n\nNow, let's move on to matrices. A matrix is a rectangular array of numbers, arranged in rows a

In [8]:
print(string["choices"][0]["message"]["content"])

Of course! I'd be happy to help you learn linear algebra. Linear algebra is a branch of mathematics that deals with vector spaces and linear transformations. It involves the study of systems of linear equations, matrices, vectors, and their properties.

To start, let's talk about vectors. A vector is a quantity that has both magnitude and direction. It can be represented geometrically as an arrow in space. In linear algebra, vectors are typically represented as column matrices. For example, the vector [1, 2, 3] can be represented as:

| 1 |
| 2 |
| 3 |

Vectors can be added together, and scalar multiplication can be applied to them. Addition of vectors is done by adding corresponding entries. Scalar multiplication is multiplying each entry of the vector by a scalar value.

Now, let's move on to matrices. A matrix is a rectangular array of numbers, arranged in rows and columns. Matrices are used to represent linear transformations and systems of linear equations. For example, a 2x3 matr

In [23]:
def gpt_test(prompt, current_convo):
    messages = [{"role": "system", "content" : "The user wants to learn about a specific topic, keep them on that topic and try to not let them veer off course. Whenever they might do so offer to open up a separate discussion instead of directly answering them"},]
    messages += current_convo
    messages += [{"role": "user", "content": inp}]
    for x in messages:
        print(x)
    response = openai.ChatCompletion.create(
        #Choose model
        model = "gpt-3.5-turbo-16k",
        #Define 
        messages = messages,
        max_tokens = 500,
        temperature = 0.8)
    return response

In [24]:
current_convo =[]
inp = ""
while (True):
    inp = input("$ ")
    if (inp == "end_conversation"):
        break
    output = gpt_test(inp, current_convo)
    output_string = output["choices"][0]["message"]["content"]
    current_convo += [{"role": "user", "content": inp}]
    current_convo += [{"role": "assistant", "content": output_string}]
    print(output_string)

$ I want to learn about linear algebra
{'role': 'system', 'content': 'The user wants to learn about a specific topic, keep them on that topic and try to not let them veer off course. Whenever they might do so offer to open up a separate discussion instead of directly answering them'}
{'role': 'user', 'content': 'I want to learn about linear algebra'}
That's great! Linear algebra is a fascinating subject. It deals with vectors, vector spaces, matrices, and linear transformations. It has wide applications in various fields like computer science, physics, engineering, and economics. What aspect of linear algebra are you interested in learning more about?
$ donkey
{'role': 'system', 'content': 'The user wants to learn about a specific topic, keep them on that topic and try to not let them veer off course. Whenever they might do so offer to open up a separate discussion instead of directly answering them'}
{'role': 'user', 'content': 'I want to learn about linear algebra'}
{'role': 'assista

In [ ]:
temp_username = "manh"
#Record schema for querying conversations:
#    every user will have a list of records associated with them (optional for now)
#    records and their relations stored in a single json file
#    every conversation (regardles of branching or not) will be stored in a different json file
"""
Schema for user record:
{'userID': 'Mike123',
'record_list':['record1','record2'],
}
Schema for record-conversation relation:
{'recordID':'record1',
'start_conversationID':'cID0',
'custom_system_message':'Teach the user linear algebra,
                         don't let them veer far from
                         the current conversation topic',
'record_keywords': ['AI', 'AI applications', 'product mangagement'],
}

Schema for conversation-conversation relations: %
{'conversationID': 'cID1',
'parentID': 'cID0'
'childrenID': ['cID2', 'cID3'],
'time_created':'1231809184',
}

Schema for conversation-correspondence relations:
{'conversationID': 'cID1',
'conversationName': 'Linear Algebra',
'conversation_keywords: ['math', 'matrices', 'eigenvalues', 'AI'], # ignore this for now
'conversation_history': [
    [{'role':'user', 'content': 'teach me linear'},
     {'role':'assistant'}
],
}
"""

In [ ]:
"""
Basic info:
- model variable defines what model to use. Be careful as different models cost differetly
- ChatCompletetion.create() has 3 main roles:
    - system: meta-prompt, basically what the software engineers want the model to do
    - user: end user input, free form. keep in mind prompt jailbreaking when designing prompts
    - assistant: used to plug in arbitrary model output. basically a gas lighting tool, also
    useful for creating conversational memory

Meta prompt:
- Defines basic behavior of model, if calling of functions or returning json output is 
desired, you have to *verbally* convince the models to use the functions. There is no
guarantee, and god knows I have no idea how openAI implements it in the back end

Function:
- Not a real function, but a mental? verbal? (who knows) algorithm for the model to undergo
when doing inference. Model will (or will not) call function depending on wording of meta prompt.
Outputs are always defined, though not necessarily will be returned (model will sometimes
empty jsons for example). To make it work, use prompt engineering.
- For reference, removing "use only the 'classify' function" from the meta prompt actually
*improved* consistency of the model using the function correctly.
- Note: There can be many functions

Todo:
- Return all model outputs and data (load time, token count, etc) to a json file for 
documentation and more r&d.

--- Everything here is under heavy experimentation, no one knows what's going on and nothing 
makes any sense ---
"""
def doorman(prompt):
    #Load meta prompt
    with open('llmlads_prompt.txt','r') as file:
        meta_prompt = file.read()
    #Define API Call
    response = openai.ChatCompletion.create(
        #Choose model
        model = "gpt-3.5-turbo-16k",
        #Define 
        messages = [
            {"role": "system", "content": meta_prompt},
            {"role": "user", "content": prompt},
        ],
        #Define 'function' for model to call during execution
        functions = [
            {
                "name": "classify",
                #Verbal description to help the model use the function correctly,
                #for the time being, telling the model to return 0 if no matches are found
                #seems to do absolutely nothing
                "description": "classify a user statement into one of the statement groups using a json format, output 0 if there are no matches",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "group": {
                            "type": "number",
                            "description": "index of group that the statement most resembles",
                        },
                    },
                },
                #This actually does absolutely nothing, model frequently returns empty json
                #anyway, needs further testing
                "required": ["group"],
            },
        ],
        #Define model parameters
        max_tokens = 100,
        temperature = 0.8,
    )
    try:
        if (response["choices"][0]["message"]["function_call"]["arguments"] != "{}"):
            return int(response["choices"][0]["message"]["function_call"]["arguments"][13:-2])
        else:
            return 0
    except:
        return 0

In [ ]:
app = Flask(__name__)
slack_event_adapter = SlackEventAdapter(
    os.environ['SIGNING_SECRET'], '/slack/events',app)

client = slack.WebClient(token=os.environ['SLACK_TOKEN'])
#response = client.api_call("auth.text")
#BOT_ID = response['user_id']

@slack_event_adapter.on('message')
def message(payload):
    event = payload.get('event',{})
    channel_id = event.get('channel')
    user_id = event.get('user')
    text = event.get('text')
    inp = "User input: " + "\"" + text + "\""
    if user_id != 'U05EY26K3FZ':
        index = doorman(inp)
        if index == 0:
            text = "Yaabaadabaadoo"
            client.chat_postMessage(channel=channel_id, text=text)
        else:
            text = answers[index - 1]
            client.chat_postMessage(channel=channel_id, text=text)
            
app.run(debug=True)

In [ ]:
# Main tesing loop, repeatedly prompts user until user inputs "end_conversation"
inp = ""
while (True):
    inp = input("$ ")
    if (inp == "end_conversation"):
        break
    #This formats the prompt to as "User input 'prompt' ", unclear if it does anything
    inp = "User input: " + "\"" + inp + "\""
    index = doorman(inp)
    if index == 0:
        print("I'm sorry but could you provide me with a bit more clarification?")
    else:
        print(answers[index - 1])

In [2]:
import slack
import openai
import os
from pathlib import Path
from dotenv import load_dotenv
from flask import Flask
from slackeventsapi import SlackEventAdapter
import pandas as pd
df = pd.read_csv('QandA_list.csv', encoding='utf-8')
answers = df["Question answers"].tolist()

env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)

openai.api_key = token=os.environ['CHAT_TOKEN']

def doorman(prompt):
    #Load meta prompt
    with open('llmlads_prompt.txt','r') as file:
        meta_prompt = file.read()
    #Define API Call
    response = openai.ChatCompletion.create(
        #Choose model
        model = "gpt-3.5-turbo-16k",
        #Define 
        messages = [
            {"role": "system", "content": meta_prompt},
            {"role": "user", "content": prompt},
        ],
        #Define 'function' for model to call during execution
        functions = [
            {
                "name": "classify",
                #Verbal description to help the model use the function correctly,
                #for the time being, telling the model to return 0 if no matches are found
                #seems to do absolutely nothing
                "description": "classify a user statement into one of the statement groups using a json format, output 0 if there are no matches",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "group": {
                            "type": "number",
                            "description": "index of group that the statement most resembles",
                        },
                    },
                },
                #This actually does absolutely nothing, model frequently returns empty json
                #anyway, needs further testing
                "required": ["group"],
            },
        ],
        #Define model parameters
        max_tokens = 100,
        temperature = 0.8,
    )
    try:
        if (response["choices"][0]["message"]["function_call"]["arguments"] != "{}"):
            return int(response["choices"][0]["message"]["function_call"]["arguments"][13:-2])
        else:
            return 0
    except:
        return 0

app = Flask(__name__)
slack_event_adapter = SlackEventAdapter(
    os.environ['SIGNING_SECRET'], '/slack/events',app)

client = slack.WebClient(token=os.environ['SLACK_TOKEN'])
#response = client.api_call("auth.text")
#BOT_ID = response['user_id']

@slack_event_adapter.on('message')
def message(payload):
    event = payload.get('event',{})
    channel_id = event.get('channel')
    user_id = event.get('user')
    text = event.get('text')
    inp = "User input: " + "\"" + text + "\""
    if user_id != 'U05EY26K3FZ':
        index = doorman(inp)
        if index == 0:
            text = "I'm sorry but could you provide me with a bit more clarification?"
            client.chat_postMessage(channel=channel_id, text=text)
        else:
            text = answers[index - 1]
            client.chat_postMessage(channel=channel_id, text=text)
            
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/usr/lib/python3/d

SystemExit: 1

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
